<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Colabora%C3%A7%C3%A3o%20de%20v%C3%A1rios%20agentes%20para%20an%C3%A1lise%20financeira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install litellm
!pip install crewai 'tools'

In [29]:
import os
from crewai import Agent, Task, Crew,  Process, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [28]:
# Suas chaves
os.environ["GROQ_API_KEY"] = "SUA_API_KEY"
os.environ ["SERPER_API_KEY"] = "SUA_API_KEY"

# LLMs
llm = LLM (
    model = 'groq/llama3-8b-8192',
    temperature=0.2,
    api_key="SUA_API_KEY"
    )

In [27]:
# Tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [26]:
# Criando os Agentes

data_analyst_agent = Agent(
    role="Analista de Dados",
    goal="Monitorar e analisar dados de mercado em tempo real "
          "para identificar tendências e prever movimentações de mercado.",
    backstory="Especializado em mercados financeiros, este agente "
              "usa modelagem estatística e aprendizado de máquina "
              "para fornecer insights cruciais. Com uma habilidade para dados, "
              "o Agente Analista de Dados é a base para "
              "informar as decisões de negociação.",
    verbose=True,
    allow_delegation=True,
    memory=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [25]:
trading_strategy_agent = Agent(
    role="Desenvolvedor de Estratégias de Negociação",
    goal="Desenvolver e testar várias estratégias de negociação com base "
          "nos insights do Agente Analista de Dados.",
    backstory="Equipado com um profundo entendimento dos "
              "mercados financeiros e análise quantitativa, este agente "
              "concebe e refina estratégias de negociação. Ele avalia "
              "o desempenho de diferentes abordagens para determinar "
              "as opções mais lucrativas e avessas ao risco.",
    verbose=True,
    allow_delegation=True,
    memory=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [24]:
execution_agent = Agent(
    role="Consultor comercial",
    goal="Sugerir estratégias ótimas de execução de negociação "
          "com base nas estratégias de negociação aprovadas.",
    backstory="Este agente se especializa em analisar o momento, o preço "
              "e os detalhes logísticos de potenciais negociações. Ao avaliar "
              "esses fatores, ele fornece sugestões bem fundamentadas sobre "
              "quando e como as negociações devem ser executadas para maximizar "
              "a eficiência e a aderência à estratégia.",
    verbose=True,
    allow_delegation=True,
    memory=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [44]:
risk_management_agent = Agent(
    role="Consultor de Risco",
    goal="Avaliar e fornecer insights sobre os riscos "
          "associados a potenciais atividades de negociação.",
    backstory="Armado com um profundo entendimento de modelos de avaliação "
              "de risco e dinâmica de mercado, este agente analisa minuciosamente "
              "os riscos potenciais das negociações propostas. Ele oferece uma "
              "análise detalhada da exposição ao risco e sugere salvaguardas "
              "para garantir que as atividades de negociação estejam alinhadas "
              "com a tolerância ao risco da empresa.",
    verbose=True,
    allow_delegation=True,
    memory=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [45]:
# Criando as Tasks

# Tarefa para o Agente Analista de Dados: Analisar Dados de Mercado
data_analysis_task = Task(
    description=(
        "Monitorar e analisar continuamente os dados de mercado para "
        "a ação selecionada ({stock_selection}). "
        "Use modelagem estatística e aprendizado de máquina para "
        "identificar tendências e prever movimentações de mercado."
    ),
    expected_output=(
        "Insights e alertas sobre oportunidades ou ameaças "
        "significativas de mercado para {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [46]:
# Tarefa para o Agente Desenvolvedor de Estratégias de Negociação: Desenvolver Estratégias de Negociação
strategy_development_task = Task(
    description=(
        "Desenvolver e refinar estratégias de negociação com base nos "
        "insights do Analista de Dados e na tolerância ao risco "
        "definida pelo usuário ({risk_tolerance}). "
        "Considere as preferências de negociação ({trading_strategy_preference})."
    ),
    expected_output=(
       "Um conjunto de potenciais estratégias de negociação para {stock_selection} "
        "que se alinhem com a tolerância ao risco do usuário."
    ),
    agent=trading_strategy_agent,
)


In [47]:
# Tarefa para o Agente Consultor comercial: Planejar a Execução de Negociação
execution_planning_task = Task(
    description=(
        "Analisar as estratégias de negociação aprovadas para determinar os "
        "melhores métodos de execução para {stock_selection}, "
        "considerando as condições de mercado atuais e o preço ideal."
    ),
    expected_output=(
        "Planos de execução detalhados sugerindo como e quando "
        "executar as negociações para {stock_selection}."
    ),
    agent=execution_agent,
)


In [48]:
# Tarefa para o Agente Consultor de Risco: Avaliar Riscos de Negociação
risk_assessment_task = Task(
    description=(
        "Avaliar os riscos associados às estratégias de negociação "
        "propostas e aos planos de execução para {stock_selection}. "
        "Fornecer uma análise detalhada dos riscos potenciais "
        "e sugerir estratégias de mitigação."
    ),
    expected_output=(
         "Um relatório abrangente de análise de risco detalhando os "
         "riscos potenciais e as recomendações de mitigação para {stock_selection}."
    ),
    agent=risk_management_agent,
    context= [execution_planning_task, strategy_development_task, data_analysis_task]
)

In [49]:
# Criando a Crew
# Definir a equipe com agentes e tarefas

financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],
    process=Process.sequential,
    verbose=True
)

In [50]:
# Dados de exemplo para iniciar o processo

financial_trading_inputs = {
    'stock_selection': 'ITUB3',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [66]:
from IPython.display import Markdown

# Access the 'raw' attribute of the CrewOutput object to get the text content.
markdown_text = result.raw

# Now, pass the markdown_text to the Markdown function.
Markdown(markdown_text)

**Risks Associated with the Proposed Negotiation Strategies for ITUB3**

Based on the approved negotiation strategies for ITUB3, considering the current market conditions and the ideal price, I have identified the following potential risks and mitigation strategies:

1. **Market Volatility Risk**: The stock price of ITUB3 is expected to fluctuate, which may impact the negotiation outcome. Mitigation strategy: Monitor market conditions closely and adjust the negotiation strategy accordingly.
2. **Stakeholder Goals and Constraints Risk**: The goals and constraints of the key stakeholders involved in the negotiation may not align, which may lead to a failed negotiation. Mitigation strategy: Identify and understand the goals and constraints of the key stakeholders and develop a negotiation strategy that takes into account their needs.
3. **Negotiation Strategy Risk**: The negotiation strategy may not be effective, leading to a failed negotiation. Mitigation strategy: Develop a comprehensive negotiation strategy that takes into account the current market conditions, the ideal price, and the goals and constraints of the key stakeholders.
4. **Communication Risk**: Poor communication between the parties involved in the negotiation may lead to misunderstandings and a failed negotiation. Mitigation strategy: Establish clear communication channels and ensure that all parties involved in the negotiation are aware of the agreed-upon terms.
5. **Time-Sensitive Risk**: The negotiation may not be completed within the desired timeframe, leading to a failed negotiation. Mitigation strategy: Set a realistic timeframe for the negotiation and ensure that all parties involved in the negotiation are aware of the deadline.

**Recommendations**

Based on the analysis of the potential risks and mitigation strategies, I recommend the following:

1. Develop a comprehensive negotiation strategy that takes into account the current market conditions, the ideal price, and the goals and constraints of the key stakeholders.
2. Monitor market conditions closely and adjust the negotiation strategy accordingly.
3. Establish clear communication channels and ensure that all parties involved in the negotiation are aware of the agreed-upon terms.
4. Set a realistic timeframe for the negotiation and ensure that all parties involved in the negotiation are aware of the deadline.

By following these recommendations, we can ensure that the negotiation is executed effectively and efficiently, and that the best possible outcome is achieved for all parties involved.